# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import numpy as np
import json

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,masterton,-40.9597,175.6575,22.71,55,83,1.59,NZ,2024-01-30 23:56:20-05:00
1,1,port-aux-francais,-49.3500,70.2167,4.07,73,88,12.95,TF,2024-01-30 23:56:20-05:00
2,2,jackson,42.4165,-122.8345,17.93,34,0,9.26,US,2024-01-30 23:56:21-05:00
3,3,filingue,14.3521,3.3168,18.63,21,8,4.95,NE,2024-01-30 23:56:21-05:00
4,4,mersa matruh,31.3525,27.2453,8.11,81,40,4.12,EG,2024-01-30 23:56:22-05:00


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size='Humidity',
    color='City',
    alpha=0.5
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
city_data_df = city_data_df[(city_data_df['Max Temp'] > 21) & (city_data_df['Max Temp'] < 27) & (city_data_df['Wind Speed'] < 4.5) & (city_data_df['Cloudiness'] == 0)]

# Drop any rows with null values
city_data_df = city_data_df.copy()
city_data_df.dropna(inplace=True)

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
144,144,port elizabeth,-33.9180,25.5701,21.84,87,0,3.09,ZA,2024-01-30 23:57:35-05:00
226,226,tura,25.5198,90.2201,23.50,32,0,1.47,IN,2024-01-30 23:58:18-05:00
237,237,kasane,-17.8167,25.1500,21.05,100,0,1.54,BW,2024-01-30 23:58:23-05:00
383,383,boa vista,2.8197,-60.6733,25.99,78,0,2.06,BR,2024-01-30 23:58:34-05:00
433,433,imperatriz,-5.5264,-47.4917,25.03,94,0,1.03,BR,2024-01-31 00:00:12-05:00


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = np.nan


# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
144,port elizabeth,ZA,-33.9180,25.5701,87,NaN
226,tura,IN,25.5198,90.2201,32,NaN
237,kasane,BW,-17.8167,25.1500,100,NaN
383,boa vista,BR,2.8197,-60.6733,78,NaN
433,imperatriz,BR,-5.5264,-47.4917,94,NaN


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Base URL
base_url = 'https://api.geoapify.com/v2/places?'


# Set parameters to search for a hotel
radius = 10000
params = {
    'categories': 'accommodation.hotel',
    'apiKey': geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row['Lat']
    longitude = row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f'circle:{longitude},{latitude},{radius}'
    params["bias"] = f'proximity:{longitude},{latitude}'
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
port elizabeth - nearest hotel: Waterford Hotel
tura - nearest hotel: No hotel found
kasane - nearest hotel: Kasane Travel Lodge
boa vista - nearest hotel: Aipana Plaza Hotel
imperatriz - nearest hotel: Hotel Posseidon
east london - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
144,port elizabeth,ZA,-33.9180,25.5701,87,Waterford Hotel
226,tura,IN,25.5198,90.2201,32,No hotel found
237,kasane,BW,-17.8167,25.1500,100,Kasane Travel Lodge
383,boa vista,BR,2.8197,-60.6733,78,Aipana Plaza Hotel
433,imperatriz,BR,-5.5264,-47.4917,94,Hotel Posseidon
460,east london,ZA,-33.0153,27.9116,88,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

hover_cols=['Hotel Name']

# Configure the map plot
map_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size='Humidity',
    color='City',
    alpha=0.6,
    hover_cols=['Hotel Name','Country']
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)